In [5]:
import pandas as pd
import datetime
df = pd.read_csv('locust_data.csv')

df.columns.unique()

Index(['Category', 'lat', 'lon', 'Part of month', 'Observation Date',
       'Impacted Area (ha)', 'Country', 'Location Name', 'Data updated on'],
      dtype='object')

In [6]:
df['Observation Date'] = pd.to_datetime(df['Observation Date'], errors='coerce')
df_2018_onwards = df[df['Observation Date'] >= '2018-01-01']

df_2018_onwards.to_csv('locust_data_2018_onwards.csv', index=False)
